In [ ]:
# %load init.py
import os
import pickle
import sys
# Enable module import from the parent directory from notebooks
sys.path.append(os.path.abspath('..'))
import time

import matplotlib as mpl
# Select plotting backend
mpl.use('nbAgg')

import matplotlib.pyplot as plt
# Customize plotting
plt.style.use('seaborn-paper')
plt.rcParams['axes.labelsize'] = 11.0
plt.rcParams['axes.titlesize'] = 12.0
plt.rcParams['errorbar.capsize'] = 3.0
plt.rcParams['figure.dpi'] = 72.0
plt.rcParams['figure.titlesize'] = 12.0
plt.rcParams['legend.fontsize'] = 10.
plt.rcParams['lines.linewidth'] = 1.
plt.rcParams['xtick.labelsize'] = 11.0
plt.rcParams['ytick.labelsize'] = 11.0

import numpy as np
import sympy as sp
sp.init_printing(euler=True, use_latex=True)

from IPython import display
from scipy import io, optimize
from sklearn import metrics

import core
import dynamicals
import kernels
import numericals
import utils

In [ ]:
dynamical = dynamicals.LotkaVolterra()

spl_t_0, spl_t_T, spl_freq = 0, 2, 100
obs_t_0, obs_t_T, obs_freq = 0, 2, 10
est_t_0, est_t_T, est_freq = 0, 2, 10
spl_tps, obs_tps, obs_t_indices, est_tps, est_t_indices = utils.create_time(
    spl_t_0, spl_t_T, spl_freq, obs_t_0, obs_t_T, obs_freq, est_t_0, est_t_T, est_freq)
X_0 = np.array([5., 3.]) 
theta = np.array([2., 1., 1., 4.]) 
rho_2 = None
phi = [
    # (Kernal name, Kernal parameters)
    ('rbf', np.sqrt([2.5, 0.02])),
    ('rbf', np.sqrt([2.5, 0.02]))
]
sigma_2 = np.array([0.1, 0.1]) 
delta = np.full(dynamical.num_x, True)
gamma = np.array([5e-3, 5e-3]) 

opt_method = 'Newton-CG'
opt_tol = 1e-6
max_init_iter = None
max_iter = 2000

plotting_enabled = False
plotting_freq = 50
plotting_config = None

In [ ]:
dynamical = dynamicals.ProteinSignallingTransduction() 

spl_t_0, spl_t_T, spl_freq = 0, 100, 20
obs_t_0, obs_t_T, obs_freq = 0, 100, 20
est_t_0, est_t_T, est_freq = 0, 100, 20
spl_tps, obs_tps, obs_t_indices, est_tps, est_t_indices = utils.create_time(
    spl_t_0, spl_t_T, spl_freq, obs_t_0, obs_t_T, obs_freq, est_t_0, est_t_T, est_freq)
t_indices = np.array([0, 1, 2, 4, 5, 7, 10, 15, 20, 30, 40, 50, 60, 80, 100]) * spl_freq
obs_tps = obs_tps[t_indices]
obs_t_indices = obs_t_indices[t_indices]
est_tps = est_tps[t_indices]
est_t_indices = est_t_indices[t_indices]
X_0 = np.array([1., 0., 1., 0., 0.]) 
theta = np.array([0.07, 0.6, 0.05, 0.3, 0.017, 3.]) 
rho_2 = None
sigma_2 = np.full(dynamical.num_x, 1e-2) 
delta = np.full(dynamical.num_x, True)
# gamma = np.full(dynamical.num_x, 5e-3) 
# gamma = np.array([1e-4, 1e-4, 2e-6, 1e-5, 5e-2])
# phi = [
#     # (Kernal name, Kernal parameters)
#     ('sigmoid', np.array([1., .4, 15.])),
#     ('sigmoid', np.array([.18, .6, 25.])),
#     ('sigmoid', np.array([.84, 3., 3.1])),
#     ('sigmoid', np.array([.62, 2., 2.1])),
#     ('sigmoid', np.array([.84, 3., 2.9])),
# ]
gamma = np.array([1e-3, 1e-3, 1e-5, 1e-4, 5e-3])
phi = [
    # (Kernal name, Kernal parameters)
    ('sigmoid', np.array([1., .4, 16.])),
    ('sigmoid', np.array([.44, .2, 8.])),
    ('sigmoid', np.array([.83, 3, 3.1])),
    ('sigmoid', np.array([.62, 1., 1.1])),
    ('sigmoid', np.array([.83, 2.9, 3])),
]

opt_method = 'Newton-CG'
opt_tol = 1e-6
max_init_iter = None
max_iter = 2000

plotting_enabled = True
plotting_freq = 50
plotting_config = {
    'x': {
        'xlim': (0, 100),
        'ylim': (0., 1.2)
    }
}

In [ ]:
dynamical = dynamicals.ProteinSignallingTransductionWithoutKm() 
spl_t_0, spl_t_T, spl_freq = 0, 100, 20
obs_t_0, obs_t_T, obs_freq = 0, 100, 20
est_t_0, est_t_T, est_freq = 0, 100, 20
spl_tps, obs_tps, obs_t_indices, est_tps, est_t_indices = utils.create_time(
    spl_t_0, spl_t_T, spl_freq, obs_t_0, obs_t_T, obs_freq, est_t_0, est_t_T, est_freq)
t_indices = np.array([0, 1, 2, 4, 5, 7, 10, 15, 20, 30, 40, 50, 60, 80, 100]) * spl_freq
obs_tps = obs_tps[t_indices]
obs_t_indices = obs_t_indices[t_indices]
est_tps = est_tps[t_indices]
est_t_indices = est_t_indices[t_indices]
X_0 = np.array([1., 0., 1., 0., 0.]) 
theta = np.array([0.07, 0.6, 0.05, 0.3, 0.017]) 
rho_2 = None
sigma_2 = np.full(dynamical.num_x, 1e-2) 
delta = np.full(dynamical.num_x, True)
delta[0] = False
# gamma = np.full(dynamical.num_x, 1e-5) 
# gamma = np.array([5e-4, 5e-3, 5e-5, 1e-4, 1e-4])
# phi = [
#     # (Kernal name, Kernal parameters)
#     ('sigmoid', np.array([1., .4, 16.])),
#     ('sigmoid', np.array([.44, .2, 8.])),
#     ('sigmoid', np.array([.83, 3, 3.1])),
#     ('sigmoid', np.array([.6, 1., 1.1])),
#     ('sigmoid', np.array([.83, 2.9, 3])),
# ]
# gamma = np.array([5e-4, 5e-3, 5e-5, 1e-4, 5e-4])
# gamma = np.array([1e-3, 1e-3, 5e-5, 1e-4, 1e-3])
# gamma = np.array([1e-3, 1e-3, 1e-5, 1e-4, 1e-3])
# gamma = np.array([1e-3, 1e-3, 1e-5, 5e-5, 1e-3])
gamma = np.array([1e-3, 1e-3, 1e-5, 1e-4, 5e-3])
phi = [
    # (Kernal name, Kernal parameters)
    ('sigmoid', np.array([1., .4, 16.])),
    ('sigmoid', np.array([.44, .2, 8.])),
    ('sigmoid', np.array([.83, 3, 3.1])),
    ('sigmoid', np.array([.62, 1., 1.1])),
    ('sigmoid', np.array([.83, 2.9, 3])),
]

opt_method = 'Newton-CG'
opt_tol = 1e-6
max_init_iter = None
max_iter = 2000

plotting_enabled = True
plotting_freq = 50
plotting_config = {
    'x': {
        'xlim': (0, 100),
        'ylim': (0., 1.2)
    }
}

In [ ]:
spl_X = dynamical.generate_sample_path(theta, rho_2, X_0, spl_tps)

In [ ]:
obs_Y = utils.collect_observations(spl_X, obs_t_indices, sigma_2, [0, None])

In [ ]:
mu, Sigma, inv_Sigma, m, inv_Lambda, Lambda = core.init_with_gaussian_processes(dynamical, 
                                                                                obs_Y, obs_tps, 
                                                                                est_tps,
                                                                                phi, sigma_2, delta, gamma,
                                                                                plotting_enabled)

utils.plot_states(dynamical, spl_X, spl_tps, obs_Y, obs_tps, delta, mu, est_tps, plotting_config)

num_x = dynamical.num_x
num_theta = dynamical.num_theta
num_est_t = est_tps.size

lpmf_pos = core.LaplaceMeanFieldPositivitySymbolic(dynamical, num_est_t, np.abs(mu), inv_Sigma, m, Lambda, debug=True)

use_X_X_tilde = True
use_X_theta_tilde = True

lpmf_pos.construct_X_objective_functions(use_X_X_tilde, use_X_theta_tilde)

use_theta_X_tilde = True
use_theta_theta_tilde = True

lpmf_pos.construct_theta_objective_functions(use_theta_X_tilde, use_theta_theta_tilde)

eta_theta, log_eta_theta, eta_X, log_eta_X = lpmf_pos.init_optimization()

figure, plotting_config = utils.create_estimation_figure(dynamical, delta)
utils.add_sample_path(figure, plotting_config, spl_X, spl_tps)
utils.add_observations(figure, plotting_config, obs_Y, obs_tps, delta)
utils.add_gaussian_regression_result(figure, plotting_config, eta_X, est_tps)

eta_theta_costs = []
eta_X_costs = []

for it in range(1, 100):
    eta_theta, eta_theta_tilde, result = lpmf_pos.optimize_theta(opt_method)
    eta_theta_cost = result.fun       
    eta_theta_costs.append(result.fun)

#     eta_theta[:] = theta
    eta_X_cost = 0
    for i in range(num_x):
        eta_X, eta_X_tilde, result = lpmf_pos.optimize_x(i, opt_method)
        eta_X_cost += result.fun
    eta_X_costs.append(eta_X_cost)        
    
    if it % 1 != 0:
        print('.', end='')
    else:
        print('Iteration: {}, theta cost: {:.4f}, X cost: {:.4f}'.format(it, eta_theta_cost, eta_X_cost))
        utils.add_estimation_step(figure, plotting_config, dynamical, eta_X, est_tps, theta, eta_theta)

utils.add_estimation_result(figure, plotting_config, dynamical, eta_X, est_tps, theta, eta_theta)
# utils.plot_estimation_result(plotting_config, dynamical, spl_X, spl_tps, obs_Y, obs_tps, delta, eta_X, None,
#                              est_tps, theta, eta_theta, None)

In [ ]:
# Helper to plot costs
figure = plt.figure(figsize=plt.figaspect(0.4))
ax = figure.add_subplot(1, 2, 1)
ax.plot(eta_X_costs)
ax.set_title('Cost for X')

ax = figure.add_subplot(1, 2, 2)
ax.plot(eta_X_costs)
ax.set_title('Cost for theta')

figure.tight_layout()
plt.show()

In [ ]:
# Helper to plot theta for protein signaling transduction
figure = plt.figure()
ax = plt.gca()
bar_width = 0.3
bar_indices = np.arange(theta.size - 1)
ax.bar(bar_indices, theta[:-1], bar_width, color='C0', label='Truth')
ax.bar(bar_indices + bar_width, eta_theta[:-1], bar_width, color='C2', label='Estimation',
#        yerr=2 * np.sqrt(np.diagonal(Xi_theta))[:-1],
       error_kw=dict(ecolor='0.2', capsize=3., capthick=1.))
ax.set_ylabel('Value')
ax.set_title('Theta')
ax.set_xticks(bar_indices + bar_width / 2)
ax.set_xticklabels([r'${}$'.format(label) for label in dynamical.theta_labels])
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles=handles, labels=labels, loc=0)
figure.tight_layout()
plt.show()